### IMPORTING LIBRARIES

In [21]:
import os
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    RocCurveDisplay,
    ConfusionMatrixDisplay,
    precision_recall_curve
)

warnings.filterwarnings("ignore")

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["font.size"] = 12

print("Libraries imported successfully.")


Libraries imported successfully.


### DATASET PREPARATION

In [22]:
# Load the dataset
try:
    df = pd.read_csv('preprocessed_dataset/preprocessed_data.csv')
    print("Dataset loaded successfully.")
except FileNotFoundError:
    raise FileNotFoundError("CSV not found. Place it in 'preprocessed_dataset/' or adjust the path.")

Dataset loaded successfully.


In [23]:
print("Shape:", df.shape)
display(df.head())

Shape: (7043, 31)


,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,-1.277445,-1.160323,-0.994242,0,False,True,False,False,True,...,False,False,False,False,False,False,True,False,True,False
1,0,0.066327,-0.259629,-0.173244,0,True,False,False,True,False,...,False,False,False,False,True,False,False,False,False,True
2,0,-1.236724,-0.362660,-0.959674,1,True,False,False,True,False,...,False,False,False,False,False,False,True,False,False,True
3,0,0.514251,-0.746535,-0.194766,0,True,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False
4,0,-1.236724,0.197365,-0.940470,1,False,False,False,True,False,...,False,False,False,False,False,False,True,False,True,False


In [24]:
# Separate features (X) and target (y)
if "Churn" not in df.columns:
    raise KeyError("Column 'Churn' not found in preprocessed data. "
                   "Ensure Task 1 saved the target as 'Churn'.")

X = df.drop("Churn", axis=1)
y = df["Churn"]

print("Feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)


Feature matrix shape: (7043, 30)
Target vector shape: (7043,)


In [25]:
# Quick class balance check
print("\nClass distribution (Churn):")
print(y.value_counts())
print("\nClass distribution (%):")
print((y.value_counts(normalize=True) * 100).round(2))


Class distribution (Churn):
Churn
0    5174
1    1869
Name: count, dtype: int64

Class distribution (%):
Churn
0    73.46
1    26.54
Name: proportion, dtype: float64


### TRAIN–TEST SPLIT

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)


Training set shape: (5634, 30)
Test set shape: (1409, 30)


### EVALUATION FUNCTION

In [ ]:
def evaluate_classifier(model, X_train, X_test, y_train, y_test, model_name="Model"):
    """
    Fit the model, predict on train and test,
    and print common classification metrics.
    """
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Probabilities for ROC-AUC (if available)
    if hasattr(model, "predict_proba"):
        y_test_proba = model.predict_proba(X_test)[:, 1]
    else:
        # Fallback using decision_function if predict_proba is unavailable
        y_test_proba = model.decision_function(X_test)
    
    metrics = {}
    
    for split, y_true, y_hat in [
        ("Train", y_train, y_train_pred),
        ("Test",  y_test,  y_test_pred)
    ]:
        acc  = accuracy_score(y_true, y_hat)
        prec = precision_score(y_true, y_hat, zero_division=0)
        rec  = recall_score(y_true, y_hat, zero_division=0)
        f1   = f1_score(y_true, y_hat, zero_division=0)
        
        if split == "Test":
            auc = roc_auc_score(y_true, y_test_proba)
        else:
            auc = np.nan  # just to keep structure
        
        metrics[f"{split}_accuracy"]  = acc
        metrics[f"{split}_precision"] = prec
        metrics[f"{split}_recall"]    = rec
        metrics[f"{split}_f1"]        = f1
        if split == "Test":
            metrics[f"{split}_roc_auc"] = auc
        
        print(f"\n{model_name} – {split} metrics")
        print(f"Accuracy : {acc:.4f}")
        print(f"Precision: {prec:.4f}")
        print(f"Recall   : {rec:.4f}")
        print(f"F1 score : {f1:.4f}")
        if split == "Test":
            print(f"ROC-AUC  : {auc:.4f}")
    
    # Confusion matrix (test set)
    cm = confusion_matrix(y_test, y_test_pred)
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"{model_name} – Confusion Matrix (Test)")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()
    
    print(f"\nClassification report – {model_name} (Test set):")
    print(classification_report(y_test, y_test_pred, digits=4))
    
    return metrics, y_test_proba
